In [1]:
from TreeCluster import *
from treeswift import read_tree_newick
from scipy.optimize import minimize
import sys
import numpy as np

In [2]:
max_iter = 45

In [3]:
class StopGridSearch( Exception ):
    pass
class StopFineGridSearch( Exception ):
    pass

In [4]:
'''
tree_file="phylogeny.nwk"
threshold_val=0.57
threshold_free=None
method='avg_clade'
support=float('-inf')
expected_num_clusters= 1000
'''

def someiter_search(tree_file="phylogeny.nwk",threshold_val=0.57,threshold_free=None,method='avg_clade',support=float('-inf'),expected_num_clusters= 1000,max_iter = 10):
    def precal_GridSearch(threshold, tree_file=tree_file, threshold_free=threshold_free, method=method, support=support):
        cluster = run_TreeCluster(threshold=threshold_val, tree_file=tree_file, threshold_free=threshold_free, method=method, support=support)
        loss = expected_num_clusters-len(cluster)
        grad = 0.1 * pow(10,-len(str(loss)))
        min_loss= expected_num_clusters

        return loss,grad,min_loss

    def GridSearch(loss,grad,min_loss,threshold, tree_file=tree_file, threshold_free=threshold_free, method=method, support=support):
        threshold_vals= [threshold]
        loss_vals = [loss]
        current_grad = [grad]
        i=1
        best_val = threshold
        curr_loss= loss
        #print("Iteration : ", 0, " Loss: ", loss, " Threshold_val:", threshold)
        try:
            while(i<=max_iter+1):
                curr_grad = ( 0.1 * pow(10,-len(str(loss_vals[i-1]))) ) 
                #print(curr_grad)
                curr_val = threshold_vals[i-1] - (grad*curr_loss)*np.sign(grad*loss)
                if curr_val<0:
                    raise StopGridSearch
                    curr_val = best_val
                    min_loss = loss_vals[i]
                curr_loss = abs(expected_num_clusters-len(run_TreeCluster(threshold=curr_val, tree_file=tree_file, threshold_free=threshold_free, method=method, support=support)))
                threshold_vals.append(curr_val)
                loss_vals.append(curr_loss)
                #print(loss_vals[i], min_loss)
                if loss_vals[i] < min_loss:
                    min_loss = loss_vals[i] 
                    best_val = curr_val
        
                #print(" Iteration : ", i, " Loss: ", curr_loss, " Threshold_val:", curr_val, "Min_loss:",min_loss )
         
                i+=1
        
        except StopGridSearch:
            pass
        
        return best_val, min_loss
    


    j=0
    
    while j<=max_iter:
        if j==0:
            loss,grad,min_loss = precal_GridSearch(threshold=threshold_val, tree_file=tree_file, threshold_free=threshold_free, method=method, support=support)
            best_min_loss = min_loss
            curr_threshold_val = threshold_val
            best_threshold_val = threshold_val
        else:
            curr_threshold_val, min_loss = GridSearch(loss,grad,min_loss,threshold=curr_threshold_val, tree_file=tree_file, threshold_free=threshold_free, method=method, support=support)
            grad = min_loss * ( 0.1 * pow(10,-len(str(min_loss))) ) * pow(.1,j)
            loss = min_loss
            if loss< best_min_loss:
                best_min_loss= min_loss
                best_threshold_val = curr_threshold_val
        
        #print("ITER:",j, " Loss: ", best_min_loss, " Threshold_val:", best_threshold_val)
        j+=1
       
    return best_min_loss, best_threshold_val

In [8]:
def search_best_thresh(threshold_val_iter,expected_num_clusters=1000,tree_file="phylogeny.nwk",threshold_free=None,method='avg_clade',support=float('-inf'),max_iter = 10):
    thresh_list = list(np.random.uniform(0,1,threshold_val_iter))

    for val in thresh_list:
        best_min_loss = expected_num_clusters
        min_loss, curr_threshold_val = someiter_search(tree_file="phylogeny.nwk",threshold_val=val,threshold_free=None,method='avg_clade',support=float('-inf'),expected_num_clusters= 1000,max_iter = 10)
        if min_loss < best_min_loss:
                best_min_loss= min_loss
                best_threshold_val = curr_threshold_val
            
    
        print("REAL ITER:",thresh_list.index(val)+1, " Loss: ", min_loss, " Threshold_val:", curr_threshold_val)
    
    return best_threshold_val

In [9]:
search_best_thresh(threshold_val_iter=3,expected_num_clusters=1000,tree_file="phylogeny.nwk",threshold_free=None,method='avg_clade',support=float('-inf'),max_iter = 10)

[0.3566814401034576, 0.4171070498085625, 0.030453822331192315]
YES
REAL ITER: 1  Loss:  23  Threshold_val: 0.0838814401034576
YES
REAL ITER: 2  Loss:  24  Threshold_val: 0.08350704980856245
YES
REAL ITER: 3  Loss:  1000  Threshold_val: 0.030453822331192315


0.08350704980856245